## Mancala AI

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
random.seed(109)

## Changed Mancala class as everything was being handled inside it. Instead just left the game logic and created new classes for every player.

In [2]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit=4):

        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player + 1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(self.board) - 2]
        self.p2_mancala_index = len(self.board) - 1

        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i + 1, player_1_pits[i], 
                        player_2_pits[-(i + 1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i + 1, player_1_pits[i], 
                        player_2_pits[-(i + 1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)

    def valid_move(self, pit):
        player1pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1] + 1]
        player2pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1] + 1]
        if (pit > self.pits_per_player or pit <= 0): 
            return False
        
        if self.current_player == 1:
            return player1pits[pit - 1] > 0
        else:
            return player2pits[pit - 1] > 0

        
    def play(self, pit):
        pitsPerPlayer = self.pits_per_player
        n = len(self.board)
        
        if(not self.valid_move(pit)):
            print("Invalid Move")
            return
        
        # Get the number of stones in the selected pit
        if self.current_player == 1:
            actual_pit = pit - 1
        else:
            actual_pit = pit + pitsPerPlayer

        stones = self.board[actual_pit]
        self.board[actual_pit] = 0

        for i in range(stones):
            current_pit = (actual_pit + i + 1) % n

            # Handle the capture rule
            if (self.current_player == 1 and current_pit < pitsPerPlayer and i == stones - 1 and 
                    self.board[current_pit] == 0 and self.board[n - current_pit - 2] != 0):
                self.board[self.p1_mancala_index] += 1 + self.board[n - current_pit - 2]
                self.board[n - current_pit - 2] = 0
                
            elif (self.current_player == 2 and pitsPerPlayer < current_pit < n - 1 and i == stones - 1 and 
                    self.board[current_pit] == 0 and self.board[n - current_pit - 2] != 0):
                self.board[self.p2_mancala_index] += 1 + self.board[n - current_pit - 2]
                self.board[n - current_pit - 2] = 0
                
            else:
                self.board[current_pit] += 1
        
        self.moves.append((self.current_player, pit))
        if self.current_player == 1:
            self.current_player = 2
        else:
            self.current_player = 1


    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        player1pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player2pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        # Reset count
        count1 = 0
        count2 = 0
        for i in player2pits:
            if (i == 0):
                count1 +=1
        # Reset count
        for i in player1pits:
            if (i == 0):
                count2 += 1
                
        if (count1 == self.pits_per_player or count2 == self.pits_per_player): return True
        else: return False

### Random Player class
#### Random VS Random

In [3]:
class RandomPlayer:
    def __init__(self, game):
        self.game = game
        
    def getMove(self):
        move = random.randint(1, self.game.pits_per_player)
        while not self.game.valid_move(move): 
            move = random.randint(1, self.game.pits_per_player)
        return move
    

In [4]:
wins = [0, 0 ,0]
totMoves = []
for i in range(100):
    game = Mancala()
    randomMoves = RandomPlayer(game)
    while not game.winning_eval():
        game.play(randomMoves.getMove())
        
    totMoves.append(len(game.moves))
    if(game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]):
        wins[0] += 1
    elif(game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]):
        wins[1] += 1
    else:
        wins[2] += 1

print(wins)

[50, 45, 5]


In [5]:
print("Random player 1 has a winrate of:", wins[0], "%")
print("Random player 2 has a winrate of:", wins[1], "%")
print("The rate of ties is:", wins[2], "%")

Random player 1 has a winrate of: 50 %
Random player 2 has a winrate of: 45 %
The rate of ties is: 5 %


In [6]:
totMoves = np.array(totMoves)
print("The average number of moves for game to end is:", np.average(totMoves))

The average number of moves for game to end is: 40.42


# Minimax player
### Points of optimization: Make a new board class or state class that can be used to determine. I currently have to instantiate copies of the player and of the game in each iteration of the recursion. Not optimal.


In [7]:
class MancalaAI:
    def __init__(self, game, plies=5):
        self.game = game
        self.plies = plies

    def utility(self):
        return self.game.board[self.game.p1_mancala_index] - self.game.board[self.game.p2_mancala_index]

    def minimax(self, depth, player):
        if depth == 0 or self.game.winning_eval():
            return self.utility(), None

        bestMove = None

        if player:
            max_eval = -np.inf
            for pit in range(1, self.game.pits_per_player + 1):
                if self.game.valid_move(pit):
                    ### Had to clone game with every ply as using play on the game would make the move on the actual board. 
                    cloned_game = copy.deepcopy(self.game)
                    cloned_game.play(pit)
                    ### Had to make a new AI player as the old one still had the base board. Not the updated one.
                    eval, _ = MancalaAI(cloned_game, self.plies).minimax(depth - 1, False)
                    if eval > max_eval:
                        max_eval = eval
                        bestMove = pit
            return max_eval, bestMove

        else:
            min_eval = np.inf
            for pit in range(1, self.game.pits_per_player + 1):
                if self.game.valid_move(pit):
                    cloned_game = copy.deepcopy(self.game)
                    cloned_game.play(pit)
                    eval, _ = MancalaAI(cloned_game, self.plies).minimax(depth - 1, True)
                    if eval < min_eval:
                        min_eval = eval
                        bestMove = pit
            return min_eval, bestMove

    def getBestMove(self):
        _, bestMove = self.minimax(self.plies, self.game.current_player == 1)
        return bestMove

In [21]:
## Code to see one game.

game = Mancala()
minMaxPlayer = MancalaAI(game)
random_player = RandomPlayer(game)

while not game.winning_eval():
    if game.current_player == 1:
        minMaxPlayer = MancalaAI(game)
        move = minMaxPlayer.getBestMove()
        print("AI chooses pit:" , move)
    else:
        move = random_player.getMove()
        print("Random Player chooses pit:" , move)

    game.play(move)
    ## Uncomment to display board
    game.display_board()


print("Game Over!")
print("Final Score: Player 1 (AI):", game.board[game.p1_mancala_index], "Player 2 (Random):", game.board[game.p2_mancala_index])


AI chooses pit: 6
P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 5 | <- 3
5 -> | 4 | 5 | <- 2
6 -> |_0_|_5_| <- 1
         1         
Turn: P2
Random Player chooses pit: 2
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 4 | 5 | <- 5
3 -> | 4 | 5 | <- 4
4 -> | 4 | 6 | <- 3
5 -> | 4 | 0 | <- 2
6 -> |_0_|_5_| <- 1
         1         
Turn: P1
AI chooses pit: 2
P1               P2
     ____1____     
1 -> | 4 | 5 | <- 6
2 -> | 0 | 5 | <- 5
3 -> | 5 | 5 | <- 4
4 -> | 5 | 6 | <- 3
5 -> | 5 | 0 | <- 2
6 -> |_0_|_0_| <- 1
         7         
Turn: P2
Random Player chooses pit: 5
P1               P2
     ____2____     
1 -> | 5 | 6 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 6 | 5 | <- 4
4 -> | 5 | 6 | <- 3
5 -> | 5 | 0 | <- 2
6 -> |_0_|_0_| <- 1
         7         
Turn: P1
AI chooses pit: 3
P1               P2
     ____2____     
1 -> | 5 | 6 | <- 6
2 -> | 1 | 0 | <- 5
3 -> | 0 | 5 | <- 4
4 -> | 6 | 6 | <- 3
5 -> | 6 | 1

In [22]:
import time

wins = [0, 0 ,0]
totMoves = []
plies = [0, 0 ,0 ,0 ,0 ,0]

for i in range(100):
    game = Mancala()
    ply = 1
    minMaxPlayer = MancalaAI(game, ply)
    plies[ply] += 1
    random_player = RandomPlayer(game)

    while not game.winning_eval():
        if game.current_player == 1:
            minMaxPlayer = MancalaAI(game)
            move = minMaxPlayer.getBestMove()
        else:
            move = random_player.getMove()
        game.play(move)
        
    totMoves.append(len(game.moves))
    if(game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]):
        wins[0] += 1
    elif(game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]):
        wins[1] += 1
    else:
        wins[2] += 1


In [26]:
print("Minimax player 1 has a winrate of:", wins[0], "%")
print("Random player 2 has a winrate of:", wins[1], "%")
print("The rate of ties is:", wins[2], "%")

Minimax player 1 has a winrate of: 100 %
Random player 2 has a winrate of: 0 %
The rate of ties is: 0 %


In [11]:
totMoves = np.array(totMoves)
print("The average number of moves for game to end is:", np.average(totMoves))

The average number of moves for game to end is: 33.5


In [12]:
for i in range(1, 6):
    print("Times", i, "plies were used:", plies[i])

Times 1 plies were used: 18
Times 2 plies were used: 23
Times 3 plies were used: 14
Times 4 plies were used: 24
Times 5 plies were used: 21


#### Minimax is definitely better than the random moves. Even if we only check one ply in the game, it is enough to have a 100% winrate against the ai as we choose any move that gives us the most points in that moment.

## Alpha Beta

In [13]:
## Separate AI PLayer

class MancalaAlphaBeta:
    def __init__(self, game, plies=5):
        self.game = game
        self.plies = plies

    def utility(self):
        return self.game.board[self.game.p1_mancala_index] - self.game.board[self.game.p2_mancala_index]

    def minimax(self, depth, player, alpha, beta):
        if depth == 0 or self.game.winning_eval():
            return self.utility(), None

        bestMove = None

        if player:
            max_eval = -np.inf
            for pit in range(1, self.game.pits_per_player + 1):
                if self.game.valid_move(pit):
                    ### Had to clone game with every ply as using play on the game would make the move on the actual board. 
                    cloned_game = copy.deepcopy(self.game)
                    cloned_game.play(pit)
                    ### Had to make a new AI player as the old one still had the base board. Not the updated one.
                    eval, _ = MancalaAlphaBeta(cloned_game, self.plies).minimax(depth - 1, False, alpha, beta)
                    if eval > max_eval:
                        max_eval = eval
                        bestMove = pit
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
            return max_eval, bestMove

        else:
            min_eval = np.inf
            for pit in range(1, self.game.pits_per_player + 1):
                if self.game.valid_move(pit):
                    cloned_game = copy.deepcopy(self.game)
                    cloned_game.play(pit)
                    eval, _ = MancalaAlphaBeta(cloned_game, self.plies).minimax(depth - 1, True, alpha, beta)
                    if eval < min_eval:
                        min_eval = eval
                        bestMove = pit
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
            return min_eval, bestMove

    def getBestMove(self):
        _, bestMove = self.minimax(self.plies, self.game.current_player == 1, -np.inf, np.inf)
        return bestMove

In [14]:
game = Mancala()
ABPlayer = MancalaAlphaBeta(game)
randomPlayer = RandomPlayer(game)

while not game.winning_eval():
    if game.current_player == 1:
        ABPlayer = MancalaAlphaBeta(game)
        move = ABPlayer.getBestMove()
        print("AI chooses pit:" , move)
    else:
        move = randomPlayer.getMove()
        print("Random Player chooses pit:" , move)

    game.play(move)
    ## Uncomment to see game
    #game.display_board()


print("Game Over!")
print("Final Score: Player 1 (AI):", game.board[game.p1_mancala_index], "Player 2 (Random):", game.board[game.p2_mancala_index])


AI chooses pit: 6
Random Player chooses pit: 5
AI chooses pit: 1
Random Player chooses pit: 3
AI chooses pit: 2
Random Player chooses pit: 2
AI chooses pit: 3
Random Player chooses pit: 3
AI chooses pit: 5
Random Player chooses pit: 6
AI chooses pit: 4
Random Player chooses pit: 2
AI chooses pit: 3
Random Player chooses pit: 4
AI chooses pit: 1
Random Player chooses pit: 6
AI chooses pit: 6
Random Player chooses pit: 5
AI chooses pit: 2
Random Player chooses pit: 6
AI chooses pit: 3
Random Player chooses pit: 4
AI chooses pit: 1
Random Player chooses pit: 3
AI chooses pit: 4
Random Player chooses pit: 4
AI chooses pit: 6
Random Player chooses pit: 2
AI chooses pit: 5
Random Player chooses pit: 2
AI chooses pit: 6
Game Over!
Final Score: Player 1 (AI): 30 Player 2 (Random): 8


### Alpha Beta 5 Plies 100 Games

In [15]:
import time

wins = [0, 0 ,0]
totMoves = []
#plies = [0, 0 ,0 ,0 ,0 ,0]

startTime = time.time()
for i in range(100):
    game = Mancala()
    ABPlayer = MancalaAlphaBeta(game, 5)
    #plies[ply] += 1
    randomPlayer = RandomPlayer(game)

    while not game.winning_eval():
        if game.current_player == 1:
            ABPlayer = MancalaAlphaBeta(game)
            move = ABPlayer.getBestMove()
        else:
            move = randomPlayer.getMove()

        game.play(move)
    
    totMoves.append(len(game.moves))
    if(game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]):
        wins[0] += 1
    elif(game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]):
        wins[1] += 1
    else:
        wins[2] += 1
        
        
endTime = time.time()
timeOfExecution = endTime - startTime
    

In [16]:
print("AlphaBeta player 1 has a winrate of:", wins[0], "%")
print("Random player 2 has a winrate of:", wins[1], "%")
print("The rate of ties is:", wins[2], "%")

AlphaBeta player 1 has a winrate of: 100 %
Random player 2 has a winrate of: 0 %
The rate of ties is: 0 %


In [17]:
totMoves = np.array(totMoves)
print("The average number of moves for game to end is:", np.average(totMoves))

The average number of moves for game to end is: 33.24


In [18]:
'''
for i in range(1, 6):
    print("Times", i, "plies were used:", plies[i])
'''

'\nfor i in range(1, 6):\n    print("Times", i, "plies were used:", plies[i])\n'

In [19]:
print("Running the 100 games took a total of:", timeOfExecution, "seconds.")

Running the 100 games took a total of: 40.862528562545776 seconds.


### Difference between the Minimax and AlphaBeta AIs
Ultimately both the Minimax AI player and the AlphaBeta AI player obtain a winrate of 100%. The difference between the two players lies in the performance. This is because the minimax AI evaluates every single branch and every single node, until either the number of plies is reached, or the game reaches it's winning state.

The AlphaBeta model is the same as the minimax with one change important change. The inclusion of alpha beta prunes non optimal branches, Minimax goes through every single node one by one and evaluates all of them. The alphabeta model instead finds that a node can return a higher value, and therefore no longer needs to evaluate the other children, lowering the amount of nodes that need to be evaluated.

### Alpha Beta 10 Plies 100 Games

In [20]:
wins = [0, 0 ,0]
totMoves = []
#plies = [0, 0 ,0 ,0 ,0 ,0]

startTime = time.time()
for i in range(100):
    game = Mancala()
    ABPlayer = MancalaAlphaBeta(game, 10)
    #plies[ply] += 1
    randomPlayer = RandomPlayer(game)

    while not game.winning_eval():
        if game.current_player == 1:
            ABPlayer = MancalaAlphaBeta(game)
            move = ABPlayer.getBestMove()
        else:
            move = randomPlayer.getMove()

        game.play(move)
    
    totMoves.append(len(game.moves))
    if(game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]):
        wins[0] += 1
    elif(game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]):
        wins[1] += 1
    else:
        wins[2] += 1
        
        
endTime = time.time()
timeOfExecution = endTime - startTime

In [23]:
print("AlphaBeta player 1 has a winrate of:", wins[0], "%")
print("Random player 2 has a winrate of:", wins[1], "%")
print("The rate of ties is:", wins[2], "%")

Random player 1 has a winrate of: 100 %
Random player 2 has a winrate of: 0 %
The rate of ties is: 0 %


In [24]:
totMoves = np.array(totMoves)
print("The average number of moves for game to end is:", np.average(totMoves))

The average number of moves for game to end is: 32.22


In [25]:
'''
for i in range(1, 6):
    print("Times", i, "plies were used:", plies[i])
'''

'\nfor i in range(1, 6):\n    print("Times", i, "plies were used:", plies[i])\n'

In [26]:
print("Running the 100 games took a total of:", timeOfExecution, "seconds.")

Running the 100 games took a total of: 39.732192516326904 seconds.


There isn't much difference between the 5 ply or 10 ply AlphaBeta AIs. While theoretically more plies should result in better decisions, the effect deminishes with the amount of plies added. Even with 5 plies we are capable of a 100 percent winrate against the random player.